<a href="https://colab.research.google.com/github/sergiomar73/nlp-google-colab/blob/main/spaCY_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m spacy download en_core_web_trf

In [121]:
!pip install spacy[transformers]
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [119]:
import spacy
from spacy import displacy

In [100]:
nlp = spacy.load("en_core_web_sm")

In [101]:
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [102]:
def ner(text):
  doc = nlp(text)
  displacy.render(doc, style="ent", jupyter=True)

In [103]:
text = """MS is an autoimmune disease. In MS, the autoimmune attack is believed to target myelin, a fatty substance found on neurons in the central nervous system (CNS). This attack damages the myelin, causing demyelination."""
ner(text)

In [52]:
text = """MS is an autoimmune disease. In MS, the autoimmune attack is believed to target myelin, a fatty substance found on neurons in the central nervous system (CNS). This attack damages the myelin, causing demyelination.
Progressive loss of neuronal function and/or neuronal death, known as neurodegeneration, may also occur.
Damage to the CNS results in the symptoms and disability associated with MS.
In MS, autoreactive lymphocytes become peripherally activated and can enter the CNS through the blood-brain barrier (BBB). Autoantigens in the CNS reactivate lymphocytes resulting in inflammation and neurodegeneration."""
ner(text)

In [53]:
text = "CIS is a first episode of neurologic symptoms caused by inflammation and demyelination in the CNS."
ner(text)

In [54]:
text = "In both studies, Kesimpta significantly lowered the ARR compared with teriflunomide."
ner(text)

In [55]:
text = "Study 1: Relative reduction 51%, Kesimpta: 0.11, Teriflunomide: 0.22, P value: <0.001"
ner(text)

In [104]:
text = "Most common adverse reactions (incidence greater than 10%) were upper respiratory tract infection, injection-related reactions, headache, and local injection-site reactions."
ner(text)

In [105]:
import json
 
with open('Corona2.json', 'r') as f:
    data = json.load(f)
    
print(data['examples'][0])

{'id': '18c2f619-f102-452f-ab81-d26f7e283ffe', 'content': "While bismuth compounds (Pepto-Bismol) decreased the number of bowel movements in those with travelers' diarrhea, they do not decrease the length of illness.[91] Anti-motility agents like loperamide are also effective at reducing the number of stools but not the duration of disease.[8] These agents should be used only if bloody diarrhea is not present.[92]\n\nDiosmectite, a natural aluminomagnesium silicate clay, is effective in alleviating symptoms of acute diarrhea in children,[93] and also has some effects in chronic functional diarrhea, radiation-induced diarrhea, and chemotherapy-induced diarrhea.[45] Another absorbent agent used for the treatment of mild diarrhea is kaopectate.\n\nRacecadotril an antisecretory medication may be used to treat diarrhea in children and adults.[86] It has better tolerability than loperamide, as it causes less constipation and flatulence.[94]", 'metadata': {}, 'annotations': [{'id': '0825a1bf-

In [ ]:
training_data = {'classes' : ['MEDICINE', "MEDICALCONDITION", "PATHOGEN"], 'annotations' : []}
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  training_data['annotations'].append(temp_dict)
  
print(training_data['annotations'][0])

In [107]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

In [ ]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("training_data.spacy") # save the docbin object

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [114]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy --gpu-id 0

ℹ Saving to output directory: .
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-01-02 16:01:22,753] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-01-02 16:01:22,764] [INFO] Pipeline: ['transformer', 'ner']
INFO:spacy:Pipeline: ['transformer', 'ner']
[2023-01-02 16:01:22,767] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2023-01-02 16:01:22,768] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining m

In [122]:
nlp_ner = spacy.load("model-best")

ValueError: ignored

In [64]:
def ner_custom(text):
  doc = nlp_ner(text)
  displacy.render(doc, style="ent", jupyter=True)

In [65]:
text = """MS is an autoimmune disease. In MS, the autoimmune attack is believed to target myelin, a fatty substance found on neurons in the central nervous system (CNS). This attack damages the myelin, causing demyelination."""
ner(text)

In [66]:
text = """MS is an autoimmune disease. In MS, the autoimmune attack is believed to target myelin, a fatty substance found on neurons in the central nervous system (CNS). This attack damages the myelin, causing demyelination."""
ner_custom(text)

In [67]:
doc1 = nlp(text)
print(doc1.ents)

()


In [68]:
text = """MS is an autoimmune disease. In MS, the autoimmune attack is believed to target myelin, a fatty substance found on neurons in the central nervous system (CNS). This attack damages the myelin, causing demyelination.
Progressive loss of neuronal function and/or neuronal death, known as neurodegeneration, may also occur.
Damage to the CNS results in the symptoms and disability associated with MS.
In MS, autoreactive lymphocytes become peripherally activated and can enter the CNS through the blood-brain barrier (BBB). Autoantigens in the CNS reactivate lymphocytes resulting in inflammation and neurodegeneration."""

In [69]:
doc2 = nlp_ner(text)
print(doc2.ents)

()


In [70]:
text = "Most common adverse reactions (incidence greater than 10%) were upper respiratory tract infection, injection-related reactions, headache, and local injection-site reactions."
ner_custom(text)